## Acquiring the data

In [17]:
#Rows skipped to avoid additional cleaning, dates arsed and date used as the index
import pandas as pd



MM = pd.read_csv('../data/data20200320.csv', skiprows=[1, 483], header= 0, 
                 na_values=0, parse_dates=['Payouts Payout Currency'], 
                 index_col='Payouts Payout Currency').rename_axis(None)

In [24]:
import glob

file = glob.glob('../data/data*.csv')
file

['../data/data20200320.csv']

## Cleaning Data

In [7]:
MM.isnull().sum().sort_values()

USD     13
AUD    156
EUR    156
CAD    157
GBP    158
HKD    225
AED    256
CHF    256
JPY    258
SGD    295
CNY    314
NZD    356
MYR    358
NOK    395
SEK    397
THB    422
ZAR    440
KRW    476
CZK    477
PEN    481
SAR    481
dtype: int64

In [14]:
MM['USD'].sum()

6716166429.024874